<a href="https://colab.research.google.com/github/ravi-gopalan/ipl_analysis/blob/master/ipl_analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [360]:
!ls -la

total 137728
drwxr-xr-x 2 root root     4096 Jun 13 01:21 .
drwxr-xr-x 1 root root     4096 Jun 13 01:20 ..
---------- 1 root root 61169075 Jun  9 00:37 ball_by_ball_details.csv
-rw-r--r-- 1 root root  1202850 Jun 13 01:21 batsman_stats.csv
-rw-r--r-- 1 root root 24027925 Jun 13 01:21 cleansed_ball_by_ball_details.csv
-rw-r--r-- 1 root root 47620585 Jun 13 01:21 full_ipl_details.csv
-rw-r--r-- 1 root root  4345524 Jun 13 01:20 ipl-2008-to-2019.zip
---------- 1 root root   176133 Jun  9 00:37 ipl_summary.csv
---------- 1 root root  2060580 Jun  9 00:37 ipl.zip
-rw-r--r-- 1 root root   106682 Jun 13 01:21 match_summary.csv
-rw-r--r-- 1 root root   291508 Jun 13 01:21 players_out_by_match_view.csv


In [361]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [362]:
cd /content

/content


In [363]:
!mkdir ipl

mkdir: cannot create directory ‘ipl’: File exists


In [364]:
cd /content/ipl

/content/ipl


In [365]:
cd /root

/root


In [366]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
!cp '/content/gdrive/My Drive/Colab_stuff/kaggle.json' /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [368]:
cd /content/ipl

/content/ipl


In [369]:
!kaggle datasets download -d ravijee/ipl-2008-to-2019 -p /content/ipl

ipl-2008-to-2019.zip: Skipping, found more recently modified local copy (use --force to force download)


In [370]:
!ls -la

total 137728
drwxr-xr-x 2 root root     4096 Jun 13 01:21 .
drwxr-xr-x 1 root root     4096 Jun 13 01:20 ..
---------- 1 root root 61169075 Jun  9 00:37 ball_by_ball_details.csv
-rw-r--r-- 1 root root  1202850 Jun 13 01:21 batsman_stats.csv
-rw-r--r-- 1 root root 24027925 Jun 13 01:21 cleansed_ball_by_ball_details.csv
-rw-r--r-- 1 root root 47620585 Jun 13 01:21 full_ipl_details.csv
-rw-r--r-- 1 root root  4345524 Jun 13 01:20 ipl-2008-to-2019.zip
---------- 1 root root   176133 Jun  9 00:37 ipl_summary.csv
---------- 1 root root  2060580 Jun  9 00:37 ipl.zip
-rw-r--r-- 1 root root   106682 Jun 13 01:21 match_summary.csv
-rw-r--r-- 1 root root   291508 Jun 13 01:21 players_out_by_match_view.csv


In [371]:
pwd

'/content/ipl'

In [0]:
!unzip -q /content/ipl/ipl-2008-to-2019.zip

replace ipl_summary.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!ls -la

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import datetime as dt

In [0]:
base_dir = Path('/content/ipl')

In [0]:
df_match_summary = pd.read_csv('ipl_summary.csv')

In [0]:
df_match_summary.info()

In [0]:
df_match_summary = df_match_summary.set_index('match_id')

In [0]:
df_match_summary.head()

In [0]:
df_match_summary['match_date'] = pd.to_datetime(df_match_summary['dates'].str[2:12])

In [0]:
df_match_summary.info()

In [0]:
df_match_summary['player_of_match'] = df_match_summary['player_of_match'].str[2:-2]
df_match_summary.drop(['by','dates'], axis=1, inplace=True)
df_match_summary.rename(columns={'decision':'toss_decision'}, inplace=True)

In [0]:
df_teams = pd.DataFrame(df_match_summary.teams.str.split(',',n=2,expand=True))
df_teams.rename(columns={0:'team_1',1:'team_2'},inplace=True)
df_teams.team_1 = df_teams.team_1.str[2:-1] 
df_teams.team_2 = df_teams.team_2.str[2:-2] 
df_teams.head()

In [0]:
df_umpires = pd.DataFrame(df_match_summary.umpires.str.split(',',n=4,expand=True))
df_umpires.rename(columns={0:'umpire_1',1:'umpire_2',2:'umpire_3',3:'umpire_4'},inplace=True)
df_umpires.umpire_1 = df_umpires.umpire_1.str[2:-1] 
df_umpires.umpire_2 = df_umpires.umpire_2.str[2:-2]
df_umpires.umpire_3 = df_umpires.umpire_3.str[2:-2]
df_umpires.umpire_4 = df_umpires.umpire_4.str[2:-2]

df_umpires.head()

In [0]:
df_match_summary = df_match_summary.join(df_teams) 
df_match_summary = df_match_summary.join(df_umpires) 
df_match_summary.drop(['teams','umpires'], axis=1, inplace=True)
df_match_summary.rename(columns={'runs':'won_by_runs','wickets':'won_by_wickets'}, inplace=True)

Map team names to abbreviations and standardize names

In [0]:
team_names = {'Rajasthan Royals':'RR',\
              'Mumbai Indians' : 'MI',\
              'Delhi Daredevils':'DD',\
              'Kolkata Knight Riders':'KKR',\
              'Kings XI Punjab':'KXP',\
              'Sunrisers Hyderabad':'SRH',\
              'Deccan Chargers':'DC-Chargers',\
              'Chennai Super Kings':'CSK',\
              'Rising Pune Supergiants':'RPS',\
              'Royal Challengers Bangalore':'RCB',\
              'Pune Warriors':'PW',\
              'Gujarat Lions':'GL',\
              'Delhi Capitals':'DC-Capitals',\
              'Rising Pune Supergiant':'RPS',\
              'Kochi Tuskers Kerala':'KTK'
             }

In [0]:
df_match_summary['winner_toss'] = df_match_summary['winner_toss'].map(team_names)

In [0]:
df_match_summary['winner'] = df_match_summary['winner'].map(team_names)
df_match_summary['team_1'] = df_match_summary['team_1'].map(team_names)
df_match_summary['team_2'] = df_match_summary['team_2'].map(team_names)

In [0]:
df_match_summary['season'] = df_match_summary['match_date'].dt.year

In [0]:
df_match_summary.head()

In [0]:
df_match_summary.to_csv('match_summary.csv')

In [0]:
df_details = pd.read_csv('ball_by_ball_details.csv')

In [0]:
df_details = df_details.drop(columns=['by','competition','dates','decision','eliminator','gender','match_type','neutral_venue',\
                                      'overs','player_of_match','city','result','teams','umpires','winner','winner_toss','runs',\
                                      'wickets','match_date','season','venue'])

In [0]:
df_details['batting_team'] = df_details['batting_team'].map(team_names)

In [0]:
df_details.head()

In [0]:
df_details['wides'] = df_details['wides'].astype('float')
df_details['noballs'] = df_details['noballs'].astype('float')
df_details['legbyes'] = df_details['legbyes'].astype('float')
df_details['byes'] = df_details['byes'].astype('float')


In [0]:
df_details.info()

In [0]:
df_fielders = pd.DataFrame(df_details.fielders.str.split(',',n=2,expand=True))


df_fielders.rename(columns={0:'fielder_1',1:'fielder_2',2:'fielder_3'},inplace=True)
df_fielders.head()


In [0]:
df_fielders[df_fielders['fielder_1'].notnull() & df_fielders['fielder_2'].notnull() & df_fielders['fielder_3'].notnull()]


In [0]:
df_fielders['fielder_1'] = df_fielders['fielder_1'].str.replace(r"\[","")
df_fielders['fielder_2'] = df_fielders['fielder_2'].str.replace(r"\[","")
df_fielders['fielder_3'] = df_fielders['fielder_3'].str.replace(r"\[","")
df_fielders['fielder_1'] = df_fielders['fielder_1'].str.replace(r"\]","")
df_fielders['fielder_2'] = df_fielders['fielder_2'].str.replace(r"\]","")
df_fielders['fielder_3'] = df_fielders['fielder_3'].str.replace(r"\]","")


df_fielders['fielder_1'] = df_fielders['fielder_1'].str.replace(r"\'","")
df_fielders['fielder_2'] = df_fielders['fielder_2'].str.replace(r"\'","")
df_fielders['fielder_3'] = df_fielders['fielder_3'].str.replace(r"\'","")

In [0]:
print(df_fielders[df_fielders['fielder_1'].notnull() & df_fielders['fielder_2'].notnull() & df_fielders['fielder_3'].notnull()].head(5))
print(df_fielders[df_fielders['fielder_1'].notnull()].head(5))

In [0]:
df_fielders.info()

In [0]:
df_details = df_details.join(df_fielders)
df_details.drop(['fielders'], axis=1, inplace=True)
df_details.head()

In [0]:
df_details.info()

In [0]:
df_details['level_1'] = df_details['level_1'].astype(str)

In [0]:
delivery = {'0.1': '1','0.2': '2', '0.3': '3', '0.4': '4', '0.5': '5', '0.6': '6', '0.7': '7', '0.8': '8', '0.9': '9',\
         '1.1': '1','1.2': '2', '1.3': '3', '1.4': '4', '1.5': '5', '1.6': '6', '1.7': '7', '1.8': '8', '1.9': '9',\
         '2.1': '1','2.2': '2', '2.3': '3', '2.4': '4', '2.5': '5', '2.6': '6', '2.7': '7', '2.8': '8', '2.9': '9',\
         '3.1': '1','3.2': '2', '3.3': '3', '3.4': '4', '3.5': '5', '3.6': '6', '3.7': '7', '3.8': '8', '3.9': '9',\
         '4.1': '1','4.2': '2', '4.3': '3', '4.4': '4', '4.5': '5', '4.6': '6', '4.7': '7', '4.8': '8', '4.9': '9',\
         '5.1': '1','5.2': '2', '5.3': '3', '5.4': '4', '5.5': '5', '5.6': '6', '5.7': '7', '5.8': '8', '5.9': '9',\
         '6.1': '1','6.2': '2', '6.3': '3', '6.4': '4', '6.5': '5', '6.6': '6', '6.7': '7', '6.8': '8', '6.9': '9',\
         '7.1': '1','7.2': '2', '7.3': '3', '7.4': '4', '7.5': '5', '7.6': '6', '7.7': '7', '7.8': '8', '7.9': '9',\
         '8.1': '1','8.2': '2', '8.3': '3', '8.4': '4', '8.5': '5', '8.6': '6', '8.7': '7', '8.8': '8', '8.9': '9',\
         '9.1': '1','9.2': '2', '9.3': '3', '9.4': '4', '9.5': '5', '9.6': '6', '9.7': '7', '9.8': '8', '9.9': '9',\
         '10.1': '1','10.2': '2', '10.3': '3', '10.4': '4', '10.5': '5', '10.6': '6', '10.7': '7', '10.8': '8', '10.9': '9',\
         '11.1': '1','11.2': '2', '11.3': '3', '11.4': '4', '11.5': '5', '11.6': '6', '11.7': '7', '11.8': '8', '11.9': '9',\
         '12.1': '1','12.2': '2', '12.3': '3', '12.4': '4', '12.5': '5', '12.6': '6', '12.7': '7', '12.8': '8', '12.9': '9',\
         '13.1': '1','13.2': '2', '13.3': '3', '13.4': '4', '13.5': '5', '13.6': '6', '13.7': '7', '13.8': '8', '13.9': '9',\
         '14.1': '1','14.2': '2', '14.3': '3', '14.4': '4', '14.5': '5', '14.6': '6', '14.7': '7', '14.8': '8', '14.9': '9',\
         '15.1': '1','15.2': '2', '15.3': '3', '15.4': '4', '15.5': '5', '15.6': '6', '15.7': '7', '15.8': '8', '15.9': '9',\
         '16.1': '1','16.2': '2', '16.3': '3', '16.4': '4', '16.5': '5', '16.6': '6', '16.7': '7', '16.8': '8', '16.9': '9',\
         '17.1': '1','17.2': '2', '17.3': '3', '17.4': '4', '17.5': '5', '17.6': '6', '17.7': '7', '17.8': '8', '17.9': '9',\
         '18.1': '1','18.2': '2', '18.3': '3', '18.4': '4', '18.5': '5', '18.6': '6', '18.7': '7', '18.8': '8', '18.9': '9',\
         '19.1': '1','19.2': '2', '19.3': '3', '19.4': '4', '19.5': '5', '19.6': '6', '19.7': '7', '19.8': '8', '19.9': '9',\
        }

In [0]:
df_details['delivery'] = df_details['level_1'].map(delivery)

In [0]:
df_details['over'] = df_details['over'].astype(str)
inning_quarter = {'1': 'Q1','2': 'Q1', '3': 'Q1', '4': 'Q1', '5': 'Q1', \
                '6': 'Q2', '7': 'Q2', '8': 'Q2', '9': 'Q2', '10': 'Q2',\
                '11': 'Q3','12': 'Q3', '13': 'Q3', '14': 'Q3', '15': 'Q3',\
                '16': 'Q4', '17': 'Q4', '18': 'Q4', '19': 'Q4', '20': 'Q4'\
                }
df_details['inning_quarter'] = df_details['over'].map(inning_quarter)

In [0]:
df_details.head()

In [0]:
df_details[df_details.player_out.notna()].head()

In [0]:
df_details['kind'].unique()

In [0]:
def attribute_catch (row):
  if row['kind'] == 'caught':
    value = row['fielder_1']
  elif row['kind'] == 'caught and bowled':
    value = row['bowler']
  else:
    value = None
  return value

df_details['catch_attributed_to'] = df_details.apply (lambda row: attribute_catch(row), axis=1)

In [0]:
def attribute_wicket (row):
    if row['kind'] == 'caught':
      value = row['bowler']
    elif row['kind'] == 'lbw':
      value = row['bowler']
    elif row['kind'] == 'stumped':
      value = row['bowler']
    elif row['kind'] == 'bowled':
      value = row['bowler']
    elif row['kind'] == 'hit wicket':
      value = row['bowler']
    elif row['kind'] == 'caught and bowled':
      value = row['bowler']
    else:
      value = None
    return value
  
df_details['wicket_attributed_to'] = df_details.apply( lambda row: attribute_wicket(row), axis = 1)

In [0]:
df_details.info()

In [0]:
df_details['wicket_attributed_to'].value_counts()


In [0]:
df_details.query('kind == "caught and bowled"').info()

In [0]:
df_details.query('kind == "caught"')['fielder_1']

In [0]:
df_details.head()

In [0]:
df_details['batsman_runs'][df_details['batsman_runs'] == 6.0].count()

In [0]:
def flag_boundary_4 (row):
    if row['batsman_runs'] == 4.0:
      value = 1
    else:
      value = None  
    return value

def flag_boundary_6 (row):
    if row['batsman_runs'] == 6.0:
      value = 1
    else:
      value = None  
    return value

def flag_dot_ball (row):
    if row['batsman_runs'] == 0.0:
      value = 1
    else:
      value = None  
    return value
  
  
def flag_1_run_ball (row):
    if row['batsman_runs'] == 1.0:
      value = 1
    else:
      value = None  
    return value
  
def flag_2_run_ball (row):
    if row['batsman_runs'] == 2.0:
      value = 1
    else:
      value = None  
    return value

def flag_3_run_ball (row):
    if row['batsman_runs'] == 3.0:
      value = 1
    else:
      value = None  
    return value
  
def flag_5_run_ball (row):
    if row['batsman_runs'] == 5.0:
      value = 1
    else:
      value = None  
    return value
  
  
df_details['4_flag'] = df_details.apply(lambda row: flag_boundary_4(row), axis=1)
df_details['6_flag'] = df_details.apply( lambda row: flag_boundary_6(row), axis = 1)
df_details['dot_ball_flag'] = df_details.apply( lambda row: flag_dot_ball(row), axis = 1)
df_details['1_run_flag'] = df_details.apply( lambda row: flag_1_run_ball(row), axis = 1)
df_details['2_run_flag'] = df_details.apply( lambda row: flag_2_run_ball(row), axis = 1)
df_details['3_run_flag'] = df_details.apply( lambda row: flag_3_run_ball(row), axis = 1)
df_details['5_run_flag'] = df_details.apply( lambda row: flag_5_run_ball(row), axis = 1)

In [0]:
df_details.kind.unique()

In [0]:
def flag_caught (row):
    if row['kind'] == 'caught':
      value = 1
    else:
      value = None  
    return value

def flag_lbw (row):
    if row['kind'] == 'lbw':
      value = 1
    else:
      value = None  
    return value

def flag_stumped (row):
    if row['kind'] == 'stumped':
      value = 1
    else:
      value = None  
    return value
  
  
def flag_bowled (row):
    if row['kind'] == 'bowled':
      value = 1
    else:
      value = None  
    return value
  
def flag_run_out (row):
    if row['kind'] == 'run out':
      value = 1
    else:
      value = None  
    return value

def flag_hit_wicket (row):
    if row['kind'] == 'hit wicket':
      value = 1
    else:
      value = None  
    return value
  
def flag_caught_and_bowled (row):
    if row['kind'] == 'caught and bowled':
      value = 1
    else:
      value = None  
    return value

def flag_retired_hurt (row):
    if row['kind'] == 'retired hurt':
      value = 1
    else:
      value = None  
    return value

def flag_obstructing_the_field (row):
    if row['kind'] == 'obstructing the field':
      value = 1
    else:
      value = None  
    return value  

  
df_details['caught_flag'] = df_details.apply(lambda row: flag_caught(row), axis=1)
df_details['lbw_flag'] = df_details.apply( lambda row: flag_lbw(row), axis = 1)
df_details['stumped_flag'] = df_details.apply( lambda row: flag_stumped(row), axis = 1)
df_details['bowled_flag'] = df_details.apply( lambda row: flag_bowled(row), axis = 1)
df_details['run_out_flag'] = df_details.apply( lambda row: flag_run_out(row), axis = 1)
df_details['hit_wicket_flag'] = df_details.apply( lambda row: flag_hit_wicket(row), axis = 1)
df_details['caught_and_bowled_flag'] = df_details.apply( lambda row: flag_caught_and_bowled(row), axis = 1)
df_details['retired_hurt_flag'] = df_details.apply( lambda row: flag_retired_hurt(row), axis = 1)
df_details['obstructing_the_field_flag'] = df_details.apply( lambda row: flag_obstructing_the_field(row), axis = 1)

In [0]:
df_details[df_details['wides'].notna()].head()

In [0]:
df_details['wide_flag'] = df_details['wides'].notna()
df_details['noball_flag'] = df_details['noballs'].notna()
df_details['legbye_flag'] = df_details['legbyes'].notna()
df_details['bye_flag'] = df_details['byes'].notna()

In [0]:
df_details.info()

In [0]:
df_details.drop(['0'], inplace=True, axis=1)

In [0]:
df_details.drop(['Unnamed: 0'], inplace=True, axis=1)

In [0]:
df_details.info()

In [0]:
df_details.head()

In [0]:
df_not_out_check = df_details[['match_id','level_0','batsman','player_out','kind']].drop_duplicates()

In [0]:
df_not_out_check.query('kind=="run out"')

In [0]:
df_not_out_check.head(5)

In [0]:
def check_out(row):
  if row['kind'] != 'NaN' and row['player_out'] == row['batsman']:
    value = row['batsman']
  elif row['kind'] !='NaN' and row['player_out'] != row['batsman']:
    value = row['player_out']
  else:
    value = None
  return value

df_not_out_check['out'] = df_not_out_check.apply(lambda row: check_out(row),axis=1)

In [0]:
df_not_out_check.info()

In [0]:
df_not_out_check.query('kind=="run out"').head()

In [0]:
df_not_out = df_not_out_check.groupby(['match_id','level_0','player_out'],as_index=False)['out'].count()

In [0]:
df_not_out.head()

In [0]:
df_not_out.tail()

In [0]:
df_not_out.player_out.nunique()

In [0]:
df_not_out.query('match_id=="336002"')

In [0]:
df_details.query('match_id=="336002" & kind =="run out"')

In [0]:
df_not_out.query('match_id =="335989"')

In [0]:
df_not_out.info()

In [0]:
df_not_out.to_csv('players_out_by_match_view.csv')

In [0]:
df_details['0_extras'].head()

In [0]:
df_details['0_extras'].value_counts()

In [0]:
df_details.to_csv('cleansed_ball_by_ball_details.csv')

In [0]:
df_combined = df_details.join(df_match_summary,on='match_id',rsuffix='_ipl_summary')

In [0]:
overs = {'0.1': '1','0.2': '1', '0.3': '1', '0.4': '1', '0.5': '1', '0.6': '1', '0.7': '1', '0.8': '1', '0.9': '1',\
         '1.1': '2','1.2': '2', '1.3': '2', '1.4': '2', '1.5': '2', '1.6': '2', '1.7': '2', '1.8': '2', '1.9': '2',\
         '2.1': '3','2.2': '3', '2.3': '3', '2.4': '3', '2.5': '3', '2.6': '3', '2.7': '3', '2.8': '3', '2.9': '3',\
         '3.1': '4','3.2': '4', '3.3': '4', '3.4': '4', '3.5': '4', '3.6': '4', '3.7': '4', '3.8': '4', '3.9': '4',\
         '4.1': '5','4.2': '5', '4.3': '5', '4.4': '5', '4.5': '5', '4.6': '5', '4.7': '5', '4.8': '5', '4.9': '5',\
         '5.1': '6','5.2': '6', '5.3': '6', '5.4': '6', '5.5': '6', '5.6': '6', '5.7': '6', '5.8': '6', '5.9': '6',\
         '6.1': '7','6.2': '7', '6.3': '7', '6.4': '7', '6.5': '7', '6.6': '7', '6.7': '7', '6.8': '7', '6.9': '7',\
         '7.1': '8','7.2': '8', '7.3': '8', '7.4': '8', '7.5': '8', '7.6': '8', '7.7': '8', '7.8': '8', '7.9': '8',\
         '8.1': '9','8.2': '9', '8.3': '9', '8.4': '9', '8.5': '9', '8.6': '9', '8.7': '9', '8.8': '9', '8.9': '9',\
         '9.1': '10','9.2': '10', '9.3': '10', '9.4': '10', '9.5': '10', '9.6': '10', '9.7': '10', '9.8': '10', '9.9': '10',\
         '10.1': '11','10.2': '11', '10.3': '11', '10.4': '11', '10.5': '11', '10.6': '11', '10.7': '11', '10.8': '11', '10.9': '11',\
         '11.1': '12','11.2': '12', '11.3': '12', '11.4': '12', '11.5': '12', '11.6': '12', '11.7': '12', '11.8': '12', '11.9': '12',\
         '12.1': '13','12.2': '13', '12.3': '13', '12.4': '13', '12.5': '13', '12.6': '13', '12.7': '13', '12.8': '13', '12.9': '13',\
         '13.1': '14','13.2': '14', '13.3': '14', '13.4': '14', '13.5': '14', '13.6': '14', '13.7': '14', '13.8': '14', '13.9': '14',\
         '14.1': '15','14.2': '15', '14.3': '15', '14.4': '15', '14.5': '15', '14.6': '15', '14.7': '15', '14.8': '15', '14.9': '15',\
         '15.1': '16','15.2': '16', '15.3': '16', '15.4': '16', '15.5': '16', '15.6': '16', '15.7': '16', '15.8': '16', '15.9': '16',\
         '16.1': '17','16.2': '17', '16.3': '17', '16.4': '17', '16.5': '17', '16.6': '17', '16.7': '17', '16.8': '17', '16.9': '17',\
         '17.1': '18','17.2': '18', '17.3': '18', '17.4': '18', '17.5': '18', '17.6': '18', '17.7': '18', '17.8': '18', '17.9': '18',\
         '18.1': '19','18.2': '19', '18.3': '19', '18.4': '19', '18.5': '19', '18.6': '19', '18.7': '19', '18.8': '19', '18.9': '19',\
         '19.1': '20','19.2': '20', '19.3': '20', '19.4': '20', '19.5': '20', '19.6': '20', '19.7': '20', '19.8': '20', '19.9': '20'
        }

In [0]:
df_combined['over'] = df_combined['level_1'].map(overs)

In [0]:
df_combined['delivery'] = df_combined['level_1'].map(delivery)

In [0]:
df_combined.info()

In [0]:
df_combined.to_csv('full_ipl_details.csv')

In [0]:
df_combined.tail()

In [0]:
def calc_batting_strike_rate(row):
  if row['net_balls_faced'] != 0:
    value = row['runs_scored']/row['net_balls_faced']
  else:
    value = 0
  return value

In [0]:
# Define the aggregation calculations
batsman_aggregations = {
    'batsman_runs': { # work on the "batsman_runs" column
        'runs_scored': 'sum',  # get the sum, and call this result 'total_duration'
        'gross_balls_faced': 'count'
    },
    '4_flag': {     # Now work on the "4_flag" column
        '4s': 'count'},
    '6_flag': {     # Now work on the "6_flag" column
        '6s': 'count'},
    'dot_ball_flag': {     # Now work on the "dot_ball_flag" column
        'dot_balls': 'count'},
    'wide_flag': {     # Now work on the "wide_flag" column
        'wides_faced': 'sum'},
    'noball_flag': {     # Now work on the "noball_flag" column
        'no_balls_faced': 'sum'}, 
    '1_run_flag': {     # Now work on the "1_run_flag" column
        '1s': 'count'},
    '2_run_flag': {     # Now work on the "2_run_flag" column
        '2s': 'count'},
    '3_run_flag': {     # Now work on the "3_run_flag" column
        '3s': 'count'},
    '5_run_flag': {     # Now work on the "5_run_flag" column
        '5s': 'count'}  

}
df_batsman = df_combined.groupby(['match_id','season',\
                                  'match_date','batsman'],as_index=False)['batsman_runs','4_flag','6_flag','dot_ball_flag',\
                                                                          '1_run_flag','2_run_flag','3_run_flag','5_run_flag',\
                                                                          'wide_flag','noball_flag'].agg(batsman_aggregations)
df_batsman['net_balls_faced'] = df_batsman['gross_balls_faced']-df_batsman['wides_faced']-df_batsman['no_balls_faced']
df_batsman['strike_rate'] = df_batsman.apply(lambda row: calc_batting_strike_rate(row), axis=1)

df_batsman.head(20)

In [0]:
df_batsman.query('net_balls_faced==0')

In [0]:
df_batsman.tail(10)

In [0]:
df_not_out.info()

In [0]:
df_batsman_teams = df_combined[['match_id','batsman','batting_team']].drop_duplicates()

In [0]:
df_batsman_teams.query('batsman=="MS Dhoni"')

In [0]:
df_batsman_teams.info()

In [0]:
df_batsman = df_batsman.merge(df_not_out,how='left',left_on=['match_id','batsman'],right_on=['match_id','player_out'])

In [0]:
df_batsman = df_batsman.merge(df_batsman_teams,how='left',left_on=['match_id','batsman'],right_on=['match_id','batsman'])

In [0]:
df_batsman.tail(10)

In [0]:
df_batsman.to_csv('batsman_stats.csv')

In [0]:
df_combined['level_1'].unique()

In [0]:
list_of_teams = list(df_batsman['batting_team'].unique())
list_of_teams

In [0]:
ipl_palette = ['#EC1C24',
 '#2E0854',
 '#DCDDDF',
 "#FFFF3C",
 '#CBA92B',
 '#E32A26',
 '#004BA0',
 '#D5D6D0',
 '#7F3F98',
 '#2F9BE3',
 '#FF822A',
 '#FB2897',
 '#F97324',
 '#00008B']

sns.set_palette(palette=ipl_palette)

In [0]:
#plt.hist(df_batsman['runs_scored'])
plt.figure(figsize=(15,15))
sns.swarmplot(data=df_batsman,x='season',y='runs_scored',hue='batting_team');

In [0]:
df_batsman['strike_rate'].mean()

In [0]:
list_of_seasons = list(df_batsman['season'].unique())
list_of_seasons

In [0]:
type(list_of_seasons[0])

In [0]:
df_batsman[df_batsman['batting_team'] == list_of_teams[0]].head()

In [0]:
team_batsman_performances = []


for team in list_of_teams:
  team_batsman_performances.append(df_batsman[df_batsman['batting_team'] == team])





In [0]:
team_batsman_seasonal_performances = []

for season in list_of_seasons:
  for team in list_of_teams:
    team_batsman_seasonal_performances.append(df_batsman[(df_batsman['batting_team'] == team) & (df_batsman['season'] == season)])

In [0]:
team_batsman_seasonal_performances[9].tail()

In [0]:
team_strikes = [x['strike_rate'].mean() for x in team_batsman_seasonal_performances]

In [0]:
team_strikes

In [0]:
plt.figure(figsize=(15,15))
plt.hist([x['strike_rate'] for x in team_batsman_performances], density=1, histtype='bar',label=[x for x in list_of_teams]);
plt.legend()

In [0]:
plt.figure(figsize=(15,15))
for i in range(len(list_of_teams)):
  g = sns.distplot(team_batsman_performances[i]['strike_rate'],rug=False,hist=False,label=list_of_teams[i])

plt.legend(title='IPL', loc='upper right')
plt.show(g)


In [0]:
plt.figure(figsize=(15,15))



sns.violinplot(data=df_batsman, x='batting_team',y='strike_rate')

#plt.legend(title='IPL', loc='upper right')
#plt.show(x)





In [0]:
list_of_teams

In [0]:
plt.figure(figsize=(15,15))
for i in range(len(list_of_teams)):
  g = sns.distplot(team_batsman_seasonal_performances[i]['strike_rate'],rug=False,hist=False,label=list_of_teams[i])

plt.legend(title='IPL-2008', loc='upper right')
plt.show(g)

## Runs being scored by season

In [0]:
#plt.plot(df_combined.groupby(['season'])['total'].sum())

## Runs by innings by season

In [0]:
plt.plot(df_combined.query('level_0 == "inning_1"').groupby(['season'])['total'].mean(),color='green')
plt.plot(df_combined.query('level_0 == "inning_2"').groupby(['season'])['total'].mean(),color='blue')
plt.show()


## Highest run scorer

In [0]:
pd.DataFrame(df_combined[['season','batsman','match_id','batsman_runs']].groupby(['season','batsman','match_id'])['batsman_runs'].sum()).sort_values(by = ['batsman_runs','season'], ascending = [False, True])

In [0]:
df_batsman.info()

In [0]:
df_batsman.head()

In [0]:
def flag_200s(row):
  if row['runs_scored'] >= 200:
    value = 1
  else:
    value = 0
  return value

def flag_100s(row):
  if (row['runs_scored'] >= 100) & (row['runs_scored'] < 200):
    value = 1
  else:
    value = 0
  return value

def flag_50s(row):
  if (row['runs_scored'] >= 50) & (row['runs_scored'] < 100):
    value = 1
  else:
    value = 0
  return value

def flag_30s(row):
  if (row['runs_scored'] >= 30) & (row['runs_scored'] < 50):
    value = 1
  else:
    value = 0
  return value


In [0]:
df_batsman['200s'] = df_batsman.apply(lambda row: flag_200s(row),axis=1)
df_batsman['100s'] = df_batsman.apply(lambda row: flag_100s(row),axis=1)
df_batsman['50s'] = df_batsman.apply(lambda row: flag_50s(row),axis=1)
df_batsman['30s'] = df_batsman.apply(lambda row: flag_30s(row),axis=1)

In [0]:
df_batsman.tail()

In [0]:
df_batsman[['season','batsman','runs_scored','net_balls_faced','100s','50s','30s','6s','5s','4s','3s','2s','1s','dot_balls','strike_rate']].sort_values(by=['runs_scored'],ascending=False).head()

In [0]:
def calc_batting_strike_rate(row):
  if row['net_balls_faced'] != 0:
    value = row['runs_scored']/row['net_balls_faced']
  else:
    value = 0
  return value

def calc_batting_avg(row):
  if row['out'] != 0:
    value = row['runs_scored']/row['out']
  else:
    value = 0
  return value

def calc_balls_per_inning(row):
  if row['out'] != 0:
    value = row['net_balls_faced']/row['out']
  else:
    value = 0
  return value

In [0]:
df_batsman_seasonal_stats = df_batsman[['season','batsman','runs_scored','net_balls_faced',\
                                        'out','100s','50s','30s','6s','5s','4s','3s','2s',\
                                        '1s','dot_balls','strike_rate']].groupby(['season','batsman'],as_index=False)['runs_scored','net_balls_faced','out',\
                                                                                          '100s','50s','30s','6s','5s','4s','3s',\
                                                                                          '2s','1s','dot_balls'].sum()

In [0]:
df_batsman_seasonal_stats['season_strike_rate'] = df_batsman_seasonal_stats.apply(lambda row: calc_batting_strike_rate(row),axis=1)
df_batsman_seasonal_stats['season_batting_avg'] = df_batsman_seasonal_stats.apply(lambda row: calc_batting_avg(row),axis=1)
df_batsman_seasonal_stats['balls_per_inning'] = df_batsman_seasonal_stats.apply(lambda row: calc_balls_per_inning(row),axis=1)

df_batsman_seasonal_stats.head()


In [0]:
df_batsman_seasonal_stats.sort_values(by=['season_batting_avg','season_strike_rate','balls_per_inning'],ascending=False).head(20)


In [0]:
df_batsman_seasonal_stats.query('batsman=="MS Dhoni"')

In [0]:
df_mvp = pd.DataFrame(df_batsman_seasonal_stats.\
                      groupby('batsman',as_index=False)\
                      ['runs_scored','net_balls_faced','out',\
                       '100s','50s','30s','6s','5s','4s','3s','2s','1s','dot_balls']\
                      .sum()\
                      .sort_values(by=['runs_scored','net_balls_faced','100s','50s','30s'],\
                                   ascending=False)['batsman']\
                      .head(10)\
                      .reset_index(drop=True))



In [0]:
list(df_mvp['batsman'])


In [0]:
df_mvp

In [0]:
mvp_list = df_mvp['batsman']

In [0]:
len(mvp_list)

In [0]:
df_mvp_comparison = df_batsman_seasonal_stats.groupby('batsman',as_index=False)['runs_scored','net_balls_faced','out',\
                                                            '100s','50s','30s','6s','5s','4s','3s',\
                                                            '2s','1s','dot_balls'].sum().query('batsman in @mvp_list') 



In [0]:
df_mvp_comparison['ipl_strike_rate'] = df_mvp_comparison.apply(lambda row: calc_batting_strike_rate(row),axis=1)
df_mvp_comparison['ipl_batting_avg'] = df_mvp_comparison.apply(lambda row: calc_batting_avg(row),axis=1)
df_mvp_comparison['ipl_balls_per_inning'] = df_mvp_comparison.apply(lambda row: calc_balls_per_inning(row),axis=1)

df_mvp_comparison

In [0]:
df_mvp_comparison = df_mvp_comparison.sort_values(by=['runs_scored','net_balls_faced','100s','50s','30s'],\
                                   ascending=False).reset_index(drop=True)

In [0]:
df_mvp_comparison

In [0]:
plt.figure(figsize=(15,12))
p1 = sns.scatterplot(x="ipl_batting_avg", y="ipl_strike_rate",\
                     hue='batsman', hue_order=mvp_list,size='runs_scored',\
                     data=df_mvp_comparison\
                     
                    )
for line in range(0,df_mvp_comparison.shape[0]):
  p1.text(df_mvp_comparison.ipl_batting_avg[line]-0.015,df_mvp_comparison.ipl_strike_rate[line]+0.005,df_mvp_comparison.batsman[line])

p1.set_title('Batting Strike rates vs Batting averages for top 10 run scorers')
plt.show(p1)



In [0]:
plt.figure(figsize=(15,12))
p2 = sns.scatterplot(x="ipl_batting_avg", y="ipl_balls_per_inning",\
                     hue='batsman', hue_order=mvp_list,size='runs_scored',\
                     data=df_mvp_comparison\
                     
                    )
for line in range(0,df_mvp_comparison.shape[0]):
  p2.text(df_mvp_comparison.ipl_batting_avg[line]-0.015,df_mvp_comparison.ipl_balls_per_inning[line]+0.005,df_mvp_comparison.batsman[line])

p2.set_title('Batting Longevity vs Batting averages for top 10 run scorers')
plt.show(p2)

In [0]:
df_mvp_comparison.ipl_batting_avg[0]

In [0]:
df_mvp_comparison.shape[0]

In [0]:
df_best_batsman = df_batsman.merge(df_mvp,how='right',left_on=['batsman'],right_on=['batsman'])

In [0]:
df_best_batsman.head()

In [0]:
plt.figure(figsize=(8,8))
sns.swarmplot(data=df_best_batsman,x='strike_rate',y='batsman').set_title('Strike Rates by player for each inning')
#sns.violinplot(data=df_best_batsman,x='strike_rate',y='batsman')

In [0]:
plt.figure(figsize=(8,8))
sns.swarmplot(data=df_best_batsman,x='runs_scored',y='batsman').set_title('Runs scored by player for each inning');

In [0]:
plt.figure(figsize=(8,8))
sns.swarmplot(data=df_best_batsman,x='net_balls_faced',y='batsman').set_title('Balls faced by player for each inning')

In [0]:
sns.barplot(data=df_mvp_comparison,x='runs_scored',y='batsman')

In [0]:
df_match_summary['player_of_match'].value_counts()

In [0]:
df_details.info()

In [0]:
df_inning_wise = df_details[['match_id','level_0','inning_quarter',\
                             'player_out','batsman_runs','total','6_flag','4_flag',\
                             'dot_ball_flag','delivery','wide_flag','noball_flag']].groupby(['match_id',\
                                                                             'level_0',\
                                                                             'inning_quarter'],as_index=False)['batsman_runs','total','player_out','delivery',\
                                                                                                               'wide_flag','noball_flag','6_flag',\
                                                                                                               '4_flag','dot_ball_flag'].agg({'batsman_runs':'sum','total':'sum',\
                                                                                                               'player_out':'count','delivery':'count',\
                                                                                                               'wide_flag':'count','noball_flag':'count',\
                                                                                                               '6_flag':'count','4_flag':'count',\
                                                                                                               'dot_ball_flag':'count'})



In [0]:
df_inning_wise.info()

In [0]:
df_unstacked = df_inning_wise.set_index(\
                                        ['match_id','inning_quarter','level_0']\
                                        ).unstack().unstack()



In [0]:
df_unstacked.info()

In [0]:
df_unstacked.columns.names

In [0]:
df_unstacked.columns.values

In [0]:
df_unstacked.columns = ['_'.join(col).strip() for col in df_unstacked.columns.values]

In [0]:
df_unstacked.columns.values

In [0]:
df_unstacked.info()

In [0]:
df_unstacked.reset_index()

In [0]:
df_match_summary.info()


In [0]:
df_match_summary.merge(df_unstacked,how='left',on='match_id')

### Individual stats

In [0]:
plt.plot(df_combined.query('batsman=="CH Gayle"').groupby(['season'])['batsman_runs'].mean(),color='green')
plt.plot(df_combined.query('batsman=="V Kohli"').groupby(['season'])['batsman_runs'].mean(),color='blue')
plt.plot(df_combined.query('batsman=="MS Dhoni"').groupby(['season'])['batsman_runs'].mean(),color='red')
plt.plot(df_combined.query('batsman=="YK Pathan"').groupby(['season'])['batsman_runs'].mean(),color='red')
plt.show()


In [0]:
df_details.query('kind=="caught"').head()

In [0]:
df_bowler = df_details[['match_id','bowler','level_0','inning_quarter','delivery',\
                        'total','batsman_runs','extras_runs','wides','noballs','byes',\
                        'legbyes','player_out','kind','wicket_attributed_to',\
                        'catch_attributed_to','over','dot_ball_flag','1_run_flag',\
                        '2_run_flag','3_run_flag','4_flag','5_run_flag',\
                        '6_flag','wide_flag','noball_flag','caught_flag',\
                        'lbw_flag','bowled_flag','stumped_flag','run_out_flag',\
                        'hit_wicket_flag','caught_and_bowled_flag',\
                        'retired_hurt_flag','obstructing_the_field_flag'
                       ]].groupby(['match_id','bowler','level_0','inning_quarter','over'],as_index=False)\
                                                                      ['delivery','total','batsman_runs','extras_runs','wides','noballs','byes',\
                                                                       'player_out','kind','legbyes','wicket_attributed_to',\
                                                                      'catch_attributed_to','dot_ball_flag','1_run_flag',\
                                                                      '2_run_flag','3_run_flag','4_flag','5_run_flag',\
                                                                      '6_flag','wide_flag','noball_flag', 'caught_flag',\
                                                                       'lbw_flag','bowled_flag','stumped_flag','run_out_flag',\
                                                                       'hit_wicket_flag','caught_and_bowled_flag',\
                                                                       'retired_hurt_flag','obstructing_the_field_flag'\
                                                                      ].agg({'delivery':'count',\
                                                                             'total':'sum',\
                                                                             'batsman_runs':'sum',\
                                                                             'extras_runs':'sum',\
                                                                             'wides':'sum',\
                                                                             'noballs':'sum',\
                                                                             'byes':'sum',\
                                                                             'legbyes':'sum',\
                                                                             'player_out':'count',\
                                                                             'kind':'count',\
                                                                             'wicket_attributed_to':'count',\
                                                                             'catch_attributed_to':'count',\
                                                                             'dot_ball_flag':'count',\
                                                                             '1_run_flag':'count',\
                                                                             '2_run_flag':'count',\
                                                                             '3_run_flag':'count',\
                                                                             '4_flag':'count',\
                                                                             '5_run_flag':'count',\
                                                                             '6_flag':'count',\
                                                                             'wide_flag':'sum',\
                                                                             'noball_flag':'sum',\
                                                                             'caught_flag':'sum',\
                                                                             'lbw_flag':'sum',\
                                                                             'bowled_flag':'sum',\
                                                                             'stumped_flag':'sum',\
                                                                             'run_out_flag':'sum',\
                                                                             'hit_wicket_flag':'sum',\
                                                                             'caught_and_bowled_flag':'sum',\
                                                                             'retired_hurt_flag':'sum',\
                                                                             'obstructing_the_field_flag':'sum'\
                                                                             
                                                                             })

In [0]:
df_details.query('(match_id=="335982") & (bowler == "AA Noffke") & (over == "4")')

In [0]:
df_bowler.info()

In [0]:
df_bowler.head()

In [0]:
df_bowler.groupby(['bowler'],\
                 as_index=False)['delivery','total','batsman_runs','extras_runs','wides','noballs','byes',\
                                 'player_out','kind','legbyes','wicket_attributed_to',\
                                 'catch_attributed_to','dot_ball_flag','1_run_flag',\
                                 '2_run_flag','3_run_flag','4_flag','5_run_flag',\
                                 '6_flag','wide_flag','noball_flag', 'caught_flag',\
                                 'lbw_flag','bowled_flag','stumped_flag','run_out_flag',\
                                 'hit_wicket_flag','caught_and_bowled_flag',\
                                 'retired_hurt_flag','obstructing_the_field_flag'].agg('sum'
                                                                                        ).sort_values(\
                                                                                                       by=['wicket_attributed_to'],\
                                                                                                       ascending=False\
                                                                                                      )

In [0]:
df_bowler.groupby(['bowler'],\
                 as_index=False)['total','batsman_runs','extras_runs','wides','noballs','byes',\
                                 'player_out','kind','legbyes','wicket_attributed_to',\
                                 'catch_attributed_to','dot_ball_flag','1_run_flag',\
                                 '2_run_flag','3_run_flag','4_flag','5_run_flag',\
                                 '6_flag','wide_flag','noball_flag', 'caught_flag',\
                                 'lbw_flag','bowled_flag','stumped_flag','run_out_flag',\
                                 'hit_wicket_flag','caught_and_bowled_flag',\
                                 'retired_hurt_flag','obstructing_the_field_flag'].agg('sum').sort_values(\
                                                                                                       by=['wicket_attributed_to'],\
                                                                                                       ascending=False\
                                                                                                      )

In [0]:
df_combined['kind'].value_counts()

In [0]:
df_combined.query('kind == "caught"')

In [0]:
#df_combined['catch_attributed_to'] = np.where(df_combined.kind.eq('caught'),df_combined.fielders, np.where(df_combined.kind.eq('caught and bowled'),df_combined.bowler,'na'))

In [0]:
df_fielders = pd.DataFrame(df_combined['fielders'].apply(pd.Series))
fielder_cols = ['fielder_1','fielder_2','fielder_3']
df_fielders.columns = fielder_cols

In [0]:
df_fielders[df_fielders.fielder_2.notnull()]


In [0]:
df_match_summary.team_1.unique()

In [0]:
df_batsman_value = df_combined[['season','match_id','match_date','batsman',\
             'batsman_runs','batting_team','total']].groupby(['season','match_id','match_date','batting_team',\
                                                              'batsman'],as_index=False)['batsman_runs'].sum()

df_team_totals = df_combined[['season','match_id','match_date',\
                              'batting_team','total']].groupby(['season','match_id','match_date','batting_team',\
                                                              ],as_index=False)['total'].sum()

In [0]:
df_batsman_value.head()

In [0]:
df_team_totals.head()

In [0]:
df_team_season_scores = df_team_totals.groupby(['season','batting_team'],as_index=False)['total'].sum()

In [0]:
df_team_season_scores

In [0]:
df_batsman_value.merge(df_team_totals,how='left',on=['season','match_id','match_date','batting_team'])

In [0]:
df_batsman_value_season = df_batsman_value.groupby(['season','batting_team','batsman'],as_index=False)['batsman_runs'].sum()

In [0]:
df_batsman_value_season.head()

In [0]:
df_batsman_value_season = df_batsman_value_season.merge(df_team_season_scores,how='left',on=['season','batting_team'])
df_batsman_value_season['batsman_value'] = df_batsman_value_season['batsman_runs']/df_batsman_value_season['total'] 
df_batsman_value_season.head()

In [0]:
df_batsman_value_season.sort_values(by=['batting_team','batsman_value','season',], ascending=[True, False, True]).query('season == "2011"')

In [0]:
df_combined.head()